# NVIDIA Meeting Recorder Agent: 02-Execute Agent


In this notebook, we’ll execute and interact with the agent we created in the previous notebook. So before running this, make sure to execute `01-initialize-and-setup.ipynb`.

We’ll be using the agent to perform several tasks, including:

- Checking our meeting schedule on Google Calendar and retrieving relevant information

- Creating a meeting recorder bot in Google Meet to capture the session

- Sending the recording to a specified email address

Along the way, we’ll explore how to persist the agent’s memory using xpander’s SDK, and see how the agent interprets our prompts to carry out the tasks.

First, let’s retrieve all the necessary credentials, such as the xpander API key, NVIDIA NIM API key, and the agent ID we created in the previous notebook.



In [1]:
import os
import datetime
import time
from dotenv import load_dotenv
from xpander_sdk import XpanderClient, LLMProvider, ToolCall, Tokens, LLMTokens, Agent
from openai import OpenAI

# Load environment variables
load_dotenv()

# Verify required API keys are available
xpander_api_key = os.environ.get("XPANDER_API_KEY")
agent_id = os.environ.get("XPANDER_AGENT_ID")
nvidia_nim_api_key = os.environ.get("NVIDIA_NIM_API_KEY")

print(agent_id)
if not all([xpander_api_key, nvidia_nim_api_key, agent_id]):
    missing = []
    if not xpander_api_key: missing.append("XPANDER_API_KEY")
    if not nvidia_nim_api_key: missing.append("NVIDIA_NIM_API_KEY")
    if not agent_id: missing.append("XPANDER_AGENT_ID")
    print(f"❗️ Missing required environment variables: {', '.join(missing)}")
else:
    print(f"✅ All required API keys loaded successfully")
    
print(f"Open the agent alogside this notebook at https://app.xpander.ai/agents/{agent_id}")

75afc87c-5f62-40ab-8c5e-261fb3f4a654
✅ All required API keys loaded successfully
Open the agent alogside this notebook at https://app.xpander.ai/agents/75afc87c-5f62-40ab-8c5e-261fb3f4a654


## Define the Meeting Agent Class

Now, let’s create our `MeetingAgent` class. This class will handle both the initialization of the agent and the logic for processing input prompts and generating responses.

For each prompt the agent receives, it processes the input and generates a response through a looped operation. Each iteration of the loop consists of the following steps:

- The LLM generates a response based on the current state.

- It evaluates whether it can complete the task directly or if it needs to call a tool (e.g., Google Calendar, xpander built-in actions, etc.).

- If a tool is needed, the agent selects the appropriate tool and provides the required input arguments.

- xpander executes the tool call and returns the result to the agent for the next iteration.

- Once the LLM determines that it has all the necessary information to answer the user’s query, it calls the `xpfinish-agent-execution-finished` function, essentially ending the loop.

In [2]:
class MeetingAgent:
    """Class for running the meeting recorder agent"""
    
    def __init__(self, nvidia_nim_api_key, xpander_api_key, agent_id):
        self.agent_id = agent_id
        self.xpander_client = XpanderClient(api_key=xpander_api_key)
        self.openai_client =  OpenAI(
            base_url = "https://integrate.api.nvidia.com/v1",
            api_key = nvidia_nim_api_key
            )
        self.agent : Agent = self.xpander_client.agents.get(agent_id=self.agent_id)

    def run(self, prompt=None, thread_id=None, max_tool_used=4):
        """Run the agent with the given prompt"""

        tools_to_exclude = list()
        task = prompt or f"Please check the status of all recorded meetings."

        print("-" * 60)
        print(f"{'Continuing conversation in thread: ' + thread_id if thread_id else 'Starting a new conversation thread'}")
        print(f"Processing Task: \n\n{task}\n\n")
        print("-" * 60)
           
        # Create task with or without thread_id
        self.agent.add_task(input=task, thread_id=thread_id if thread_id else None)
        self.agent.add_messages([{"role": "assistant", "content": "the time now is : " + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}])

        # Initialize token tracking and timing
        execution_tokens = Tokens(worker=LLMTokens(completion_tokens=0, prompt_tokens=0, total_tokens=0))
        execution_start_time = time.perf_counter()
        
        
        # Run the agent until it's finished
        while not self.agent.is_finished():

         
            # Track start time for this inference
            start_time = time.perf_counter()

            if len(tools_to_exclude) != 0:
                tools = [item for item in self.agent.get_tools(llm_provider=LLMProvider.NVIDIA_NIM)
                            if item.get("function", {}).get("name") not in tools_to_exclude
                        ]
            else:
                tools = self.agent.get_tools(llm_provider=LLMProvider.NVIDIA_NIM)
            
            try:
                    # Get response from OpenAI and process
                    response = self.openai_client.chat.completions.create(
                        model="meta/llama-3.1-405b-instruct",
                        messages=[self.agent.messages[0]] + self.agent.messages[-15:] if len(self.agent.messages) > 15 else self.agent.messages,
                        tools=tools,
                        tool_choice=self.agent.tool_choice,
                        temperature=0.0,
                        max_tokens=2048,

                    )

                    # Track token usage
                    execution_tokens.worker.completion_tokens += response.usage.completion_tokens
                    execution_tokens.worker.prompt_tokens += response.usage.prompt_tokens
                    execution_tokens.worker.total_tokens += response.usage.total_tokens

            except:
                    
                    response = self.openai_client.chat.completions.create(
                        model="meta/llama-3.1-405b-instruct",
                        messages=[self.agent.messages[0]] + self.agent.messages[-15:] if len(self.agent.messages) > 15 else self.agent.messages,
                        tools=self.agent.get_tools(llm_provider=LLMProvider.NVIDIA_NIM),
                        tool_choice={"type": "function", "function": {"name": "xpfinish-agent-execution-finished"}},
                        temperature=0.0,
                        max_tokens=2048,
                    )

                    # Track token usage
                    execution_tokens.worker.completion_tokens += response.usage.completion_tokens
                    execution_tokens.worker.prompt_tokens += response.usage.prompt_tokens
                    execution_tokens.worker.total_tokens += response.usage.total_tokens
                
            # Report LLM usage to Xpander
            self.agent.report_llm_usage(
                llm_response=response.model_dump(),
                llm_inference_duration=time.perf_counter() - start_time,
                llm_provider=LLMProvider.NVIDIA_NIM
            )

            self.agent.add_messages(response.model_dump())
            # Extract and execute any tool calls
            tool_calls = XpanderClient.extract_tool_calls(
                llm_response=response.model_dump(),
                llm_provider=LLMProvider.NVIDIA_NIM
            )
            
            for tool_call in tool_calls:
                print("AI decided to call : ", tool_call.name)
                print("With the following payload : ", tool_call.payload)

            
            if tool_calls:
                print("Executing tool calls")
                responses = self.agent.run_tools(tool_calls=tool_calls)
                for response in responses:
                    print("Tool call function name : ", response.function_name)
                    print("HTTP Status code : ", response.status_code) 
                    if str(response.status_code) == "200":
                        tools_to_exclude.append(response.function_name)         
        

        # Process results
        execution_end_time = time.perf_counter()
        result = self.agent.retrieve_execution_result()
        thread_id = result.memory_thread_id
        print(f"Your thread ID is: {thread_id}")
        result_text = self.agent.messages[-1]["content"]
    
        # Report execution metrics to Xpander
        self.agent.report_execution_metrics(
            llm_tokens=execution_tokens,
            ai_model="meta/llama-3.1-405b-instruct"
        )
        print(f"Status: {result.status}")
        print(f"Result: {result_text}")
        print(f"Execution duration: {execution_end_time - execution_start_time:.2f} seconds")
    
        # Return both the result text and thread ID
        return result_text, thread_id

## Initialize the Meeting Agent

Now that we’ve defined the logic for how the agent should process user prompts, let’s go ahead and initialize our meeting agent using the `MeetingAgent` class we just created.

We also need to provide the agent with clear instructions, a defined role, and specific goals. This ensures that:

- It understands its task and objective

- It follows the correct workflow when handling user requests

- It knows which tools to use in various situations to effectively complete a task

In [3]:
# Initialize the meeting agent
meeting_agent = MeetingAgent(
    nvidia_nim_api_key=nvidia_nim_api_key,
    xpander_api_key=xpander_api_key,
    agent_id=agent_id
)

meeting_agent.agent.instructions.general = f"""
IMPORTANT: Always use and call `XpanderMessagingServiceSendEmailWithContent`when the user asks you to send assets to an email address. Make sure that you include the video and transcription link in the email, so that the recipient can download the recording assets by clicking the link.
IMPORTANT: Always use `XpanderMeetingManagementCheckRecorderStatus` everytime the user asks to check the current status of the recording bot. If the recording session has been completed, this action retrieves the link of the recorded meeting.
Be as detailed and helpful as possible when generating the response to the user.
When calling `CalendarEventManagementGetCalendarEventsById`, use only the following keys in your queryParams: ['timeMin', 'timeMax', 'singleEvents', 'orderBy']. Specify the time in the same format as the following example: '2025-04-16T00:00:00Z'
You can access the user’s calendar but cannot schedule events.
Use `CalendarEventManagementGetCalendarEventsById`to get the desired event based on the ID of the user's calendar.
Use the calendar to find meeting URLs if not provided.
To start a recording, you must have both the meeting URL and the meeting time in UTC.
Use `XpanderMeetingManagementCreateRerdingBot to create a recording bot. Make sure that you really create the recording bot in the meeting URL and also tell the user the recorder bot ID!
This will return a recording bot ID.
Check status of the recording bot using the bot ID info (The bot ID is either provided or already fetched from previous context).
Retrieve the transcript and video also using the bot ID info.
Do not execute any function unless it’s strictly required or explicitly requested by the user.
"""

meeting_agent.agent.instructions.role = """
You are a helpful meeting recorder AI agent with access to Google Calendar and Meeting Recording tools.
"""

meeting_agent.agent.instructions.goal = """
Your goal is to help users answer their query, record their meetings, check the status of the recording tools, and get the assets of the recorded meeting using the recording bot id.
"""

print(f"Meeting agent initialized with ID: {meeting_agent.agent_id}")

b'{"stdout":"bG9hZGluZyBhZ2VudCA3NWFmYzg3Yy01ZjYyLTQwYWItOGM1ZS0yNjFmYjNmNGE2NTQK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCA3NWFmYzg3Yy01ZjYyLTQwYWItOGM1ZS0yNjFmYjNmNGE2NTQgd2l0aCBleGVjdXRpb24gOTllNTYwMjItZWQyYy00YTY4LWFmOWEtNzVkMzBhMWZmZGY3Cg=="}\n'


Meeting agent initialized with ID: 75afc87c-5f62-40ab-8c5e-261fb3f4a654


## Example Use Case 1: Test Agent's Capabilities

Once we have setup everything, let's test the agent capabilities by using the following prompt:

In [4]:
prompt = """
Hi! What can you do?
"""

result, thread_id = meeting_agent.run(prompt=prompt)

------------------------------------------------------------
Starting a new conversation thread
Processing Task: 


Hi! What can you do?



------------------------------------------------------------
AI decided to call :  xpfinish-agent-execution-finished
With the following payload :  {'bodyParams': {'result': 'I can help users answer their query, record their meetings, check the status of the recording tools, and get the assets of the recorded meeting using the recording bot id.', 'is_success': True, 'agent_builder_agent_id': '02e491f0-d42f-4241-ba76-632e5f792011', 'agent_builder_ai_employee_id': '02e491f0-d42f-4241-ba76-632t12792011'}, 'queryParams': {}, 'pathParams': {}, 'headers': {}}
Executing tool calls
Tool call function name :  xpfinish-agent-execution-finished
HTTP Status code :  200
Your thread ID is: 82f824d6-5b1a-480d-b6fb-bda591a2c5b8
Status: ExecutionStatus.COMPLETED
Result: "I can help users answer their query, record their meetings, check the status of the recording to

## Example Use Case 2: Retrieve Upcoming Calendar Events

Since our agent now has access to Google Calendar, we can ask it to retrieve upcoming meetings from our schedule. To execute the agent with memory persistence, use the thread ID retrieved from the previous interaction and pass it as an argument along with your prompt.

In [4]:
# Use the same thread_id to maintain context 
prompt = """
Please list my scheduled meetings for today.
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Starting a new conversation thread
Processing Task: 


Please list my scheduled meetings for today.



------------------------------------------------------------
AI decided to call :  CalendarEventManagementGetCalendarEventsById
With the following payload :  {'bodyParams': {}, 'queryParams': {'timeMin': '2025-04-21T00:00:00Z', 'timeMax': '2025-04-21T23:59:59Z', 'singleEvents': True, 'orderBy': 'startTime'}, 'pathParams': {'calendarId': 'primary'}, 'headers': {}}
Executing tool calls
Tool call function name :  CalendarEventManagementGetCalendarEventsById
HTTP Status code :  200
AI decided to call :  xpfinish-agent-execution-finished
With the following payload :  {'bodyParams': {'result': 'You have one scheduled meeting for today: Sales Meeting at 21:30:00+02:00. The meeting URL is: https://meet.google.com/srq-owyu-jor', 'is_success': True}, 'queryParams': {}, 'pathParams': {}, 'headers': {}}
Executing tool calls
Tool call fu

As you can see, the agent called `CalendarEventManagementGetCalendarEventsById` to retrieve the ID of the relevant event from our Google Calendar. Once it had all the necessary information, it called `xpfinish-agent-execution-finished` to signal that it was ready to generate a response for the user.

After completing the task, xpander generates a thread ID for the agent's run. This thread ID is crucial for maintaining conversation history. It allows the agent to understand context and handle follow-up questions related to a previous query. We can simply include the same thread ID in subsequent prompts.

## Example Use Case 3: Set Up Meeting Recorder

We can also ask our agent to set up a recording bot for a particular meeting with prompt similar to this:

In [5]:
prompt = """
Can you record the meeting scheduled today? Please give me the recorder ID as well.
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: 74191cdf-dcc7-40b6-adbf-dfccb1432413
Processing Task: 


Can you record the meeting scheduled today? Please give me the recorder ID as well.



------------------------------------------------------------
AI decided to call :  CalendarEventManagementGetCalendarEventsById
With the following payload :  {'bodyParams': {}, 'queryParams': {'timeMin': '2025-04-21T00:00:00Z', 'timeMax': '2025-04-21T23:59:59Z', 'singleEvents': True, 'orderBy': 'startTime'}, 'pathParams': {'calendarId': 'primary'}, 'headers': {}}
Executing tool calls
Tool call function name :  CalendarEventManagementGetCalendarEventsById
HTTP Status code :  200
AI decided to call :  XpanderMeetingManagementCreateRecordingBot
With the following payload :  {'bodyParams': {'bot_name': 'Sales Meeting Recorder', 'meeting_url': 'https://meet.google.com/srq-owyu-jor', 'scheduled_time': '2025-04-21T21:30:00+02:00'}, 'queryParams': {}, 'pathP

('"The meeting recording bot has been created successfully. The recorder ID is d30bff02-0721-4a82-8c70-114f7c8805c4"',
 '74191cdf-dcc7-40b6-adbf-dfccb1432413')

The agent called the `XpanderMeetingManagementCreateRecordingBot` function to create and set up a recorder for the appropriate meeting. It first fetched the meeting link, then used that link as input when initializing the recording bot. 

## Example Use Case 4: Check the Meeting Recorder Status

This time, we’ll ask the agent for the status of the meeting recorder bot it created in the previous interaction. To do this, we’ll use the same thread ID from earlier runs to maintain the agent’s memory and context.

In [7]:
prompt = """
Please check the current status of the meeting recorder bot.
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: 74191cdf-dcc7-40b6-adbf-dfccb1432413
Processing Task: 


Please check the current status of the meeting recorder bot.



------------------------------------------------------------
AI decided to call :  CalendarEventManagementGetCalendarEventsById
With the following payload :  {'bodyParams': {}, 'queryParams': {'timeMin': '2025-04-21T00:00:00Z', 'timeMax': '2025-04-21T23:59:59Z', 'singleEvents': True, 'orderBy': 'startTime'}, 'pathParams': {'calendarId': 'primary'}, 'headers': {}}
Executing tool calls
Tool call function name :  CalendarEventManagementGetCalendarEventsById
HTTP Status code :  200
AI decided to call :  XpanderMeetingManagementCheckRecorderStatus
With the following payload :  {'bodyParams': {}, 'queryParams': {}, 'pathParams': {'recorder_id': 'd30bff02-0721-4a82-8c70-114f7c8805c4'}, 'headers': {}}
Executing tool calls
Tool call function name :  XpanderMeetingManagementCheckRec

('"The recording bot has finished recording the meeting. The video and transcript are available at the following links: https://links.xpander.ai/s2b8gy3 and https://links.xpander.ai/9yfp0y4"',
 '74191cdf-dcc7-40b6-adbf-dfccb1432413')

To complete the task, the agent called the `XpanderMeetingManagementCheckRecorderStatus` function to retrieve the current status of the recorder bot. Note that to use this function, the agent must provide the recorder bot ID, which it retrieved during the previous interaction.

In the example above, since the recorder bot has finished recording the meeting, the recording assets, such as the video and transcript, are now available for download.



## Example Use Case 5: Send Recording Assets via Email

In the last interaction, the `XpanderMeetingManagementCheckRecorderStatus` returns meeting assets such as the meeting video and transcript after it has finished recording. We can ask the agent to send these assets via email by specifying the email address in our prompt as follows:

In [8]:
prompt = """
Please send the link of the recording video and transcript to this email address: marcellusruben@gmail.com.
Write a short and professional letter in the email body.
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: 74191cdf-dcc7-40b6-adbf-dfccb1432413
Processing Task: 


Please send the link of the recording video and transcript to this email address: marcellusruben@gmail.com.
Write a short and professional letter in the email body.



------------------------------------------------------------
AI decided to call :  CalendarEventManagementGetCalendarEventsById
With the following payload :  {'bodyParams': {}, 'queryParams': {'timeMin': '2025-04-21T00:00:00Z', 'timeMax': '2025-04-21T23:59:59Z', 'singleEvents': True, 'orderBy': 'startTime'}, 'pathParams': {'calendarId': 'primary'}, 'headers': {}}
Executing tool calls
Tool call function name :  CalendarEventManagementGetCalendarEventsById
HTTP Status code :  200
AI decided to call :  XpanderMessagingServiceSendEmailWithContent
With the following payload :  {'bodyParams': {'to': ['marcellusruben@gmail.com'], 'subject': 'Meeting Recording Assets', 'body_htm

('"I have finished my job successfully. I have sent the recording assets to the email address marcellusruben@gmail.com."',
 '74191cdf-dcc7-40b6-adbf-dfccb1432413')

As you can see, our agent called the `XpanderMessagingServiceSendEmailWithContent` function to complete the task. If you head over to the email that you specified in the prompt, you'll receive an email from the agent containing the link to the video recording as well as the transcript.

You can also inspect the workflow of your agent from xpander Workbench by clicking on the "Activity" view at the top of the graph visualization of your agent.

![img_7](images/img_7.png)

There you can inspect several types of information via two key components:

- Thread: A thread is created every time you assign a task or input prompt to an agent. It includes information such as the total number of tokens generated and the creation timestamp.

- Log: This contains a detailed breakdown of your agent's execution steps. You'll see the agent's responses, any tools it called, the payload used for those tool calls, and the responses received after each call.


## Next Steps

In this blueprint, we've created a meeting recorder agent with the help of xpander.ai and NVIDIA NIM. You can now explore more use cases by integrating your agent with different SaaS platforms using xpander.ai.

You can also explore and build the agent with more powerful LLMs to ensure more accurate tool calling and response generation results.